In [7]:
import os
from PIL import Image
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np

folder_path = "/Users/Desktop/Mobile-Captured Pharmaceutical Medication Packages"

def convert_images_to_rgb(folder_path, output_folder, size=(256, 256)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert('RGB')
            img = img.resize(size)
            img.save(os.path.join(output_folder, filename))

# Пример использования
convert_images_to_rgb('Clarinaze 14 repetabs', 'Clarinaze 14 repetabs_converted')
convert_images_to_rgb('Claritine 20 tablets', 'Claritine 20 tablets_converted')

def compare_images(img1, img2):
    # Преобразуем изображения в серые
    gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)
    
    # Вычисляем SSIM
    score, _ = ssim(gray1, gray2, full=True)
    return score * 100  # Приводим к процентам

def load_image(image_path):
    return cv2.imread(image_path)

# Сравнение выбранного изображения с другими
def compare_selected_image(selected_image_path, folder_path):
    selected_image = load_image(selected_image_path)
    results = {}
    
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            current_image = load_image(img_path)
            similarity_score = compare_images(selected_image, current_image)
            results[filename] = similarity_score
    
    return results

# Пример использования
similarity_results = compare_selected_image('iphone xs max 47_converted.jpg', 'iphone xs max 358_converted.jpg')
print(similarity_results)

def highlight_differences(img1, img2):
    difference = cv2.absdiff(img1, img2)
    gray_diff = cv2.cvtColor(difference, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray_diff, 30, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Фильтрация по площади
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img1, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return img1

# Пример использования
selected_image = load_image('iphone xs max 47_converted.jpg')
other_image = load_image('iphone xs max 358_converted.jpg')
highlighted_image = highlight_differences(selected_image, other_image)

cv2.imshow("Differences", highlighted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

def generate_report(similarity_results):
    with open('report.txt', 'w') as f:
        f.write("Сравнительный отчет\n")
        f.write("===================\n")
        for filename, score in similarity_results.items():
            f.write(f"{filename}: {score:.2f}% схожести\n")

# Пример использования
generate_report(similarity_results)


FileNotFoundError: [Errno 2] No such file or directory: 'Clarinaze 14 repetabs'